In [58]:
import pandas as pd
import numpy as np

# load data
DATA_FILE = 'data/train.csv'
TEST_FILE = 'data/test.csv'
df = pd.read_csv(DATA_FILE)
TRAIN_DATA_CNT = len(df)
test_df = pd.read_csv('data/test.csv')

print df.head()
print df.shape

y_data_df = df['SalePrice']
print y_data_df.shape
x_data_df = df.drop('SalePrice',axis=1)
all_df = x_data_df.copy()
all_data_df = all_df.append(test_df) # combine train and test for encode labeler
all_data_df = all_data_df.apply(lambda x:x.fillna(x.value_counts().index[0])) # fillna with most frequent item
print x_data_df.shape
print all_data_df.shape
print all_data_df.head()

tmp_list = []
for v,c in zip(all_data_df.values[0],all_data_df.columns):
    try:
        float(v)
    except:
        print v
        tmp_list.append(c)
print tmp_list


   Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0   1          60       RL         65.0     8450   Pave   NaN      Reg   
1   2          20       RL         80.0     9600   Pave   NaN      Reg   
2   3          60       RL         68.0    11250   Pave   NaN      IR1   
3   4          70       RL         60.0     9550   Pave   NaN      IR1   
4   5          60       RL         84.0    14260   Pave   NaN      IR1   

  LandContour Utilities    ...     PoolArea PoolQC Fence MiscFeature MiscVal  \
0         Lvl    AllPub    ...            0    NaN   NaN         NaN       0   
1         Lvl    AllPub    ...            0    NaN   NaN         NaN       0   
2         Lvl    AllPub    ...            0    NaN   NaN         NaN       0   
3         Lvl    AllPub    ...            0    NaN   NaN         NaN       0   
4         Lvl    AllPub    ...            0    NaN   NaN         NaN       0   

  MoSold YrSold  SaleType  SaleCondition  SalePrice  
0      2   2008     

In [59]:
# data preprocessing
# should be done by read description 
cls_cols = ['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 
            'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 
            'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 
            'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 
            'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 
            'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 
            'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition'] 
print cls_cols
print len(cls_cols)
x_col_df = all_data_df[cls_cols] 
x_num_df = all_data_df.drop(cls_cols,axis=1)
print 'col df',x_col_df.shape
print 'num df',x_num_df.shape

# label encode
from sklearn import preprocessing
label_trans_list = []
x_col_data_list = []
for col in cls_cols:
    label_encoder = preprocessing.LabelEncoder()
    col_data = label_encoder.fit_transform(x_col_df[col])
    label_trans_list.append(label_encoder)
    x_col_data_list.append(col_data)


x_col_data_new = np.array(x_col_data_list).transpose()
final_all_data = np.hstack((x_num_df.values,x_col_data_new))
print final_all_data.shape
print '-'*20
print 'trans',final_all_data[0]
print 'org',all_data_df.values[0]
print '-'*20
final_x_data = final_all_data[:TRAIN_DATA_CNT]
final_x_test_data = final_all_data[TRAIN_DATA_CNT:]
print 'train x data',final_x_data.shape
print 'test x data',final_x_test_data.shape
final_y_data = y_data_df.values
print 'train y data',final_y_data.shape

['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']
43
col df (2919, 43)
num df (2919, 37)
(2919, 80)
--------------------
trans [  1.00000000e+00   6.00000000e+01   6.50000000e+01   8.45000000e+03
   7.00000000e+00   5.00000000e+00   2.00300000e+03   2.00300000e+03
   1.96000000e+02   7.06000000e+02   0.00000000e+00   1.50000000e+02
   8.56000000e+02   8.56000000e+02   8.54000000e+02   0.00000000e+00
   1.71000000e+03   1.00000000e+00   0.00000000e+00   2.00000000e+0

In [ ]:
# try random forest model
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn import metrics
import time
X_train, X_test, y_train, y_test = train_test_split(final_x_data, final_y_data, test_size=0.5, random_state=2333)
print np.mean(final_y_data)
print np.var(final_y_data)

import math

#A function to calculate Root Mean Squared Logarithmic Error (RMSLE)
def rmsle(y, y_pred):
    y = np.clip(y,0,max(y))
    terms_to_sum = [(math.log(y_pred[i] + 1) - math.log(y[i] + 1)) ** 2.0 for i,pred in enumerate(y_pred)]
    return (sum(terms_to_sum) * (1.0/len(y))) ** 0.5

def test_model(model):
    print '-'*20
    model.fit(X_train,y_train)
    # test
    res = model.predict(X_test)
    print 'mae', metrics.mean_absolute_error(res, y_test),'rmsle',rmsle(res,y_test),

    # test on all
    s = time.time()
    res = model.predict(final_x_data)
    print '{} data time cost {}'.format(len(final_x_data), time.time() - s),
    print 'mae', metrics.mean_absolute_error(res, final_y_data),'rmsle',rmsle(res,y_test)




model = RandomForestRegressor()
test_model(model)

for m in [LinearRegression(),Lasso(),Ridge()]:
    test_model(m)

In [90]:
print 'check xgboost'

from xgboost import XGBRegressor
model = XGBRegressor()
test_model(model)

model = XGBRegressor(n_estimators=90)
test_model(model)

model = XGBRegressor(n_estimators=90,max_depth=2)
test_model(model)

model = XGBRegressor(n_estimators=90,max_depth=4)
test_model(model)

model = XGBRegressor(n_estimators=90,max_depth=8)
test_model(model)

model = XGBRegressor(n_estimators=90,subsample=0.7)
test_model(model)

model = XGBRegressor(n_estimators=90,subsample=0.8)
test_model(model)

model = XGBRegressor(n_estimators=90,subsample=0.9)
test_model(model)

model = XGBRegressor(n_estimators=90,learning_rate=0.15)
test_model(model)

model = XGBRegressor(n_estimators=90,learning_rate=0.4)
test_model(model)

check xgboost
--------------------
mae 17677.229848 rmsle 0.146806877886 1460 data time cost 0.00998616218567 mae 13218.4435761 rmsle 0.399016670952
--------------------
mae 17728.7221104 rmsle 0.147661082825 1460 data time cost 0.0218489170074 mae 13394.1248395 rmsle 0.39802541977
--------------------
mae 18864.9193814 rmsle 0.159487211328 1460 data time cost 0.0322670936584 mae 15629.4785665 rmsle 0.393400978046
--------------------
mae 17906.0289705 rmsle 0.145553916382 1460 data time cost 0.0109059810638 mae 12182.539001 rmsle 0.398653029505
--------------------
mae 19053.1885916 rmsle 0.152238351492 1460 data time cost 0.0391719341278 mae 10170.1175969 rmsle 0.398674701766
--------------------
mae 18216.4333369 rmsle 0.1473246057 1460 data time cost 0.0081958770752 mae 13674.8023063 rmsle 0.399893548164
--------------------
mae 17722.4755298 rmsle 0.146533444637 1460 data time cost 0.0369219779968 mae 13294.3920457 rmsle 0.398943445477
--------------------
mae 17982.1415668 rmsle 

In [72]:
# only try num features
def test_num_f():
    new_final_x = x_num_df.values[:TRAIN_DATA_CNT]
    X_train, X_test, y_train, y_test = train_test_split(new_final_x, final_y_data, test_size=0.5, random_state=2333)
    print np.mean(final_y_data)
    print np.var(final_y_data)

    def test_model(model):
        print '-'*20
        model.fit(X_train,y_train)
        # test
        res = model.predict(X_test)
        print 'mae', metrics.mean_absolute_error(res, y_test)
        print 'rmsle',rmsle(res,y_test)

        # test on all
        s = time.time()
        res = model.predict(new_final_x)
        print '{} data time cost {}'.format(len(new_final_x), time.time() - s)
        print 'mae', metrics.mean_absolute_error(res, final_y_data)
        print 'rmsle',rmsle(res,y_test)
        
    model = RandomForestRegressor()
    test_model(model)

    for m in [LinearRegression(),Lasso(),Ridge()]:
        test_model(m)

    model = XGBRegressor()
    test_model(model)

test_num_f()

180921.19589
6306788585.35
--------------------
mae 21232.1346575
rmsle 0.168269133848
1460 data time cost 0.00863909721375
mae 14339.2122603
rmsle 0.39033794569
--------------------
mae 22256.4641767
rmsle 0.452247218022
1460 data time cost 0.000507116317749
mae 20693.0421242
rmsle 0.547260893519
--------------------
mae 22256.4579732
rmsle 0.452233408622
1460 data time cost 0.000272035598755
mae 20692.8643703
rmsle 0.547249752877
--------------------
mae 22252.4155397
rmsle 0.452053419175
1460 data time cost 0.000247955322266
mae 20687.6797611
rmsle 0.547124130889
--------------------
mae 18616.5887093
rmsle 0.153279113463
1460 data time cost 0.00742793083191
mae 13957.691326
rmsle 0.398859848096


In [84]:
# run on test data and save to csv
final_model = XGBRegressor(n_estimators=200,max_depth=8)
final_model.fit(final_x_data,final_y_data)
final_y = final_model.predict(final_x_test_data)
print final_y
res_df = test_df[['Id']]
res_df['SalePrice'] = final_y
print res_df.head()
res_df.to_csv('my_res.csv',index=False)


[ 123519.3828125  161995.96875    189139.84375   ...,  156956.890625
  111552.875      209626.421875 ]
     Id      SalePrice
0  1461  123519.382812
1  1462  161995.968750
2  1463  189139.843750
3  1464  191408.078125
4  1465  195680.500000


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [91]:
# try grid search
from sklearn.grid_search import GridSearchCV

X_train, X_test, y_train, y_test = train_test_split(final_x_data, final_y_data, test_size=0.5, random_state=2333)
params = {'n_estimators':list(range(20,100,10)),
          'max_depth':[2,3,4],
          'subsample':[0.8,0.9,1],
          'min_child_weight':[1,1.2,1.4],
          'learning_rate':[0.02,0.04,0.06,1]
         }
xgb = XGBRegressor()
reg = GridSearchCV(estimator=xgb,param_grid=params)
reg.fit(X_train,y_train)
print reg.best_estimator_
def test_model_on_data(model):
    print '-'*20
    # model.fit(X_train,y_train)
    print 'new_test'
    # test
    res = model.predict(X_test)
    print 'mae', metrics.mean_absolute_error(res, y_test),'rmsle',rmsle(res,y_test)

    # test on all
    s = time.time()
    res = model.predict(final_x_data)
    print '{} data time cost {}'.format(len(final_x_data), time.time() - s)
    print 'mae', metrics.mean_absolute_error(res, final_y_data),'rmsle',rmsle(res,y_test)
test_model_on_data(reg)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.06, max_delta_step=0, max_depth=4,
       min_child_weight=1.2, missing=None, n_estimators=90, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=0.8)
--------------------
new_test
mae 18343.317637 rmsle 0.152079160881
1460 data time cost 0.00905704498291
mae 13386.8074914 rmsle 0.393269414085


In [92]:
# run on test data and save to csv
final_model = XGBRegressor(n_estimators=90,max_depth=4,subsample=0.8,min_child_weight=1.2,learning_rate=0.06)
final_model.fit(X_train,y_train)
final_y = final_model.predict(final_x_test_data)
print final_y
res_df = test_df[['Id']]
res_df['SalePrice'] = final_y
print res_df.head()
res_df.to_csv('my_res_with_param_part_train.csv',index=False)


[ 123901.0078125  149877.359375   172387.28125   ...,  161057.984375
  119433.2578125  224425.375    ]
     Id      SalePrice
0  1461  123901.007812
1  1462  149877.359375
2  1463  172387.281250
3  1464  185498.000000
4  1465  195009.828125


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [93]:
# try grid search
from sklearn.grid_search import GridSearchCV

X_train, X_test, y_train, y_test = train_test_split(final_x_data, final_y_data, test_size=0.5, random_state=2333)
params = {'n_estimators':list(range(20,100,10)),
          'max_depth':[2,3,4],
          'subsample':[0.5,0.7,0.8],
          'min_child_weight':[1,1.2,1.4],
          'learning_rate':[0.02,0.04,0.06,1]
         }
xgb = XGBRegressor()
reg = GridSearchCV(estimator=xgb,param_grid=params)
reg.fit(final_x_data,final_y_data)
print reg.best_estimator_
def test_model_on_data(model):
    print '-'*20
    # model.fit(X_train,y_train)
    print 'new_test'
    # test
    res = model.predict(X_test)
    print 'mae', metrics.mean_absolute_error(res, y_test),'rmsle',rmsle(res,y_test)

    # test on all
    s = time.time()
    res = model.predict(final_x_data)
    print '{} data time cost {}'.format(len(final_x_data), time.time() - s)
    print 'mae', metrics.mean_absolute_error(res, final_y_data),'rmsle',rmsle(res,y_test)
test_model_on_data(reg)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.06, max_delta_step=0, max_depth=4,
       min_child_weight=1.2, missing=None, n_estimators=90, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=0.5)
--------------------
new_test
mae 12165.9572613 rmsle 0.105716001824
1460 data time cost 0.00998997688293
mae 11818.1129468 rmsle 0.392780553177


In [94]:
# run on test data and save to csv
final_model = XGBRegressor(n_estimators=90,max_depth=4,subsample=0.5,min_child_weight=1.2,learning_rate=0.06)
final_model.fit(final_x_data,final_y_data)
final_y = final_model.predict(final_x_test_data)
print final_y
res_df = test_df[['Id']]
res_df['SalePrice'] = final_y
print res_df.head()
res_df.to_csv('my_res_param_with_all_train.csv',index=False)

[ 124112.109375  157087.859375  180422.296875 ...,  171020.0625
  123691.96875   233536.3125  ]
     Id      SalePrice
0  1461  124112.109375
1  1462  157087.859375
2  1463  180422.296875
3  1464  180038.578125
4  1465  199115.718750


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
